In [1]:
import sys
# Verificar si se esta trabajando en el entorno virtual
print(sys.executable)


c:\Users\yohan\AppData\Local\Programs\Python\Python311\python.exe


In [2]:
# Instalar  ultima version y complemento
# %pip install -U polars
# %pip install xlsx2csv


In [3]:
import polars as pl

# Ruta del archivo de texto
ruta_historicos = './Tickets/Tickets Historico.txt'

# Lista de nombres de columnas a seleccionar
columnas_seleccionadas = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion','Fecha Termino','Fecha Cierre']

# Leer y seleccionar
df_historico = pl.read_csv(ruta_historicos,separator=";")
df_historico = df_historico.select(columnas_seleccionadas)

# Convertir las columnas 'str' a date
df_historico = df_historico.cast({"Fecha Creacion":pl.Date, "Fecha Termino":pl.Date, "Fecha Cierre":pl.Date}
)

# Renombrar columnas
df_historico = df_historico.select(
    pl.col("Numero Ticket").alias("TicketID"), *df_historico.drop("Numero Ticket")
)


print(df_historico)


shape: (9_206, 7)
┌──────────────┬──────────────┬──────────────┬─────────┬──────────────┬──────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado  ┆ Fecha        ┆ Fecha        ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---     ┆ Creacion     ┆ Termino      ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str     ┆ ---          ┆ ---          ┆ ---         │
│              ┆              ┆              ┆         ┆ date         ┆ date         ┆ date        │
╞══════════════╪══════════════╪══════════════╪═════════╪══════════════╪══════════════╪═════════════╡
│ WO0000004122 ┆ AREQUIPA -   ┆ Zona Norte   ┆ Cerrado ┆ 2022-06-01   ┆ 2022-06-14   ┆ 2022-06-14  │
│ 687          ┆ 215000       ┆              ┆         ┆              ┆              ┆             │
│ WO0000004122 ┆ METRO AV.    ┆ Zona Centro  ┆ Cerrado ┆ 2022-06-01   ┆ 2022-06-17   ┆ 2022-06-17  │
│ 649          ┆ WIESSE -     ┆              ┆         ┆              ┆  

In [4]:
ruta_actual = './Tickets/Tickets Actual.csv'

# Leer y seleccionar
df_actual = pl.read_csv(ruta_actual,separator="|")
df_actual = df_actual.select(columnas_seleccionadas)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ Numero       ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ Ticket       ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ ---          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│ str          ┆              ┆              ┆          ┆ str          ┆ str         ┆ str         │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 10/01/2023  ┆ 13/01/2023  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2/01/2023   ┆ 5/01/2023   │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [5]:
# Convertir las columnas 'str' a date

df_actual = df_actual.with_columns(pl.col("Fecha Cierre").str.split('/').apply(
        lambda lista: lista[2]+"-"+lista[1]+"-"+lista[0] if lista is not None else None
    ),
    pl.col("Fecha Termino").str.split('/').apply(
        lambda lista: lista[2]+"-"+lista[1]+"-"+lista[0] if lista is not None else None
    )
)
df_actual = df_actual.cast({"Fecha Creacion":pl.Date, "Fecha Termino":pl.Date, "Fecha Cierre":pl.Date}, strict=False)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ Numero       ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ Ticket       ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ ---          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│ str          ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

C:\Users\yohan\AppData\Local\Temp\ipykernel_20896\2305387383.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df_actual = df_actual.with_columns(pl.col("Fecha Cierre").str.split('/').apply(
C:\Users\yohan\AppData\Local\Temp\ipykernel_20896\2305387383.py:6: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("Fecha Termino").str.split('/').apply(


In [6]:
# Renombrar columnas
df_actual = df_actual.select(
    pl.col("Numero Ticket").alias("TicketID"), *df_actual.drop("Numero Ticket")
)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│              ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [7]:
# Filtrar 
df_actual= df_actual.filter(pl.col("TicketID").str.slice(0, 2)=="WO")
print(df_actual)

shape: (13_022, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│              ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [8]:
# Unir y/o concatenar lasdos bases
df_tickets = pl.concat([df_historico, df_actual])
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000412268…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-14,2022-06-14
"""WO000000412264…","""METRO AV. WIES…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-17,2022-06-17
"""WO000000412250…","""NICOLAS AYLLON…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412251…","""JAUREGUI - 405…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-18,2022-06-18
"""WO000000412274…","""CANTO GRANDE -…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412251…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-03,2022-06-03
"""WO000000412252…","""MINI AGENCIA B…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-07,2022-06-07
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-10,2022-06-10
"""WO000000412236…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-13,2022-06-13


In [9]:
# Eliminar duplicados
df_tickets= df_tickets.group_by("*").agg(pl.col("*").unique())
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000412268…","""LA SALLE - 191…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412692…","""DEL AIRE - 193…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-10,2022-06-10
"""WO000000412465…","""PLAZA CIVICA V…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-12,2022-06-12
"""WO000000412443…","""VENEZUELA - 19…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-11,2022-06-11
"""WO000000412467…","""EL PEDREGAL - …","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-20,2022-06-20
"""WO000000412531…","""MONEY AND TOUR…","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-12,2022-06-12
"""WO000000412520…","""LAS MALVINAS -…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-11,2022-06-11
"""WO000000412627…","""ANTUNEZ DE MAY…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-24,2022-06-24
"""WO000000412382…","""MOQUEGUA - 430…","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-21,2022-06-21


In [10]:
# Quitar duplicados por condicional
df_tickets=df_tickets.sort("Fecha Creacion", descending = False)
df_tickets_sin_duplicados_condiconal= df_tickets.group_by("TicketID").agg(pl.col("Fecha Creacion").last())
df_tickets_sin_duplicados_condiconal

TicketID,Fecha Creacion
str,date
"""WO000000489092…",2023-01-14
"""WO000000524014…",2023-05-18
"""WO000000558727…",2023-09-09
"""WO000000533284…",2023-06-19
"""WO000000529557…",2023-06-08
"""WO000000480912…",2022-12-13
"""WO000000554863…",2023-08-28
"""WO000000567330…",2023-10-07
"""WO000000545373…",2023-07-31


In [11]:
# Conservar las otras columnas
df_tickets = df_tickets.join(df_tickets_sin_duplicados_condiconal, on=["TicketID","Fecha Creacion"], how="inner")
df_tickets


TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000489092…","""MEGA PLAZA 2 -…","""Zona Centro""","""Cerrado""",2023-01-14,2023-01-16,2023-01-19
"""WO000000524014…","""MUSA - 193050""","""Zona Sur""","""Cerrado""",2023-05-18,2023-05-23,2023-05-27
"""WO000000558727…","""ANDAHUAYLAS - …","""Zona Norte""","""Cerrado""",2023-09-09,2023-09-22,2023-09-25
"""WO000000533284…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-06-19,2023-07-03,2023-07-06
"""WO000000529557…","""LA ENSENADA - …","""Zona Centro""","""Cerrado""",2023-06-08,2023-06-08,2023-06-11
"""WO000000480912…","""EL AGUSTINO - …","""Zona Centro""","""Cerrado""",2022-12-13,2022-12-18,2022-12-21
"""WO000000554863…","""VENEZUELA - 19…","""Zona Sur""","""Cerrado""",2023-08-28,2023-09-02,2023-09-05
"""WO000000567330…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-10-07,2023-10-11,2023-10-14
"""WO000000545373…","""CAQUETA - 1910…","""Zona Centro""","""Cerrado""",2023-07-31,2023-07-31,2023-08-03


In [12]:
# Separar la columna ubicacion
df_tickets=df_tickets.with_columns((pl.col("Ubicacion").str.split('-').map_elements(
        lambda lista: lista[0]
    )).alias("Agencia"),
    (pl.col("Ubicacion").str.split('-').map_elements(
        lambda lista: lista[1].strip()
    )).alias("AgenciaID")
)
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID
str,str,str,str,date,date,date,str,str
"""WO000000489092…","""MEGA PLAZA 2 -…","""Zona Centro""","""Cerrado""",2023-01-14,2023-01-16,2023-01-19,"""MEGA PLAZA 2 ""","""191040"""
"""WO000000524014…","""MUSA - 193050""","""Zona Sur""","""Cerrado""",2023-05-18,2023-05-23,2023-05-27,"""MUSA ""","""193050"""
"""WO000000558727…","""ANDAHUAYLAS - …","""Zona Norte""","""Cerrado""",2023-09-09,2023-09-22,2023-09-25,"""ANDAHUAYLAS ""","""205000"""
"""WO000000533284…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-06-19,2023-07-03,2023-07-06,"""CHIMPU OCLLO ""","""191113"""
"""WO000000529557…","""LA ENSENADA - …","""Zona Centro""","""Cerrado""",2023-06-08,2023-06-08,2023-06-11,"""LA ENSENADA ""","""191149"""
"""WO000000480912…","""EL AGUSTINO - …","""Zona Centro""","""Cerrado""",2022-12-13,2022-12-18,2022-12-21,"""EL AGUSTINO ""","""191041"""
"""WO000000554863…","""VENEZUELA - 19…","""Zona Sur""","""Cerrado""",2023-08-28,2023-09-02,2023-09-05,"""VENEZUELA ""","""191026"""
"""WO000000567330…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-10-07,2023-10-11,2023-10-14,"""CHIMPU OCLLO ""","""191113"""
"""WO000000545373…","""CAQUETA - 1910…","""Zona Centro""","""Cerrado""",2023-07-31,2023-07-31,2023-08-03,"""CAQUETA ""","""191091"""


In [13]:
# Cambiaratipo entero

df_tickets = df_tickets.cast({"AgenciaID": pl.Int64})
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID
str,str,str,str,date,date,date,str,i64
"""WO000000489092…","""MEGA PLAZA 2 -…","""Zona Centro""","""Cerrado""",2023-01-14,2023-01-16,2023-01-19,"""MEGA PLAZA 2 """,191040
"""WO000000524014…","""MUSA - 193050""","""Zona Sur""","""Cerrado""",2023-05-18,2023-05-23,2023-05-27,"""MUSA """,193050
"""WO000000558727…","""ANDAHUAYLAS - …","""Zona Norte""","""Cerrado""",2023-09-09,2023-09-22,2023-09-25,"""ANDAHUAYLAS """,205000
"""WO000000533284…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-06-19,2023-07-03,2023-07-06,"""CHIMPU OCLLO """,191113
"""WO000000529557…","""LA ENSENADA - …","""Zona Centro""","""Cerrado""",2023-06-08,2023-06-08,2023-06-11,"""LA ENSENADA """,191149
"""WO000000480912…","""EL AGUSTINO - …","""Zona Centro""","""Cerrado""",2022-12-13,2022-12-18,2022-12-21,"""EL AGUSTINO """,191041
"""WO000000554863…","""VENEZUELA - 19…","""Zona Sur""","""Cerrado""",2023-08-28,2023-09-02,2023-09-05,"""VENEZUELA """,191026
"""WO000000567330…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-10-07,2023-10-11,2023-10-14,"""CHIMPU OCLLO """,191113
"""WO000000545373…","""CAQUETA - 1910…","""Zona Centro""","""Cerrado""",2023-07-31,2023-07-31,2023-08-03,"""CAQUETA """,191091


In [14]:
# Crear la nueva columna by condicion
df_tickets = df_tickets.with_columns(
    pl.when(pl.col("Fecha Termino").is_null()).then(pl.col("Fecha Cierre")).otherwise(pl.col("Fecha Termino")).alias("Fecha Real Fin")
)
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin
str,str,str,str,date,date,date,str,i64,date
"""WO000000489092…","""MEGA PLAZA 2 -…","""Zona Centro""","""Cerrado""",2023-01-14,2023-01-16,2023-01-19,"""MEGA PLAZA 2 """,191040,2023-01-16
"""WO000000524014…","""MUSA - 193050""","""Zona Sur""","""Cerrado""",2023-05-18,2023-05-23,2023-05-27,"""MUSA """,193050,2023-05-23
"""WO000000558727…","""ANDAHUAYLAS - …","""Zona Norte""","""Cerrado""",2023-09-09,2023-09-22,2023-09-25,"""ANDAHUAYLAS """,205000,2023-09-22
"""WO000000533284…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-06-19,2023-07-03,2023-07-06,"""CHIMPU OCLLO """,191113,2023-07-03
"""WO000000529557…","""LA ENSENADA - …","""Zona Centro""","""Cerrado""",2023-06-08,2023-06-08,2023-06-11,"""LA ENSENADA """,191149,2023-06-08
"""WO000000480912…","""EL AGUSTINO - …","""Zona Centro""","""Cerrado""",2022-12-13,2022-12-18,2022-12-21,"""EL AGUSTINO """,191041,2022-12-18
"""WO000000554863…","""VENEZUELA - 19…","""Zona Sur""","""Cerrado""",2023-08-28,2023-09-02,2023-09-05,"""VENEZUELA """,191026,2023-09-02
"""WO000000567330…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-10-07,2023-10-11,2023-10-14,"""CHIMPU OCLLO """,191113,2023-10-11
"""WO000000545373…","""CAQUETA - 1910…","""Zona Centro""","""Cerrado""",2023-07-31,2023-07-31,2023-08-03,"""CAQUETA """,191091,2023-07-31


In [15]:
# Calculamos diferencia de fechas en dias
df_tickets = df_tickets.with_columns(( pl.col("Fecha Real Fin") - pl.col("Fecha Creacion") ).dt.total_days().alias("Dias Cierre"))

df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre
str,str,str,str,date,date,date,str,i64,date,i64
"""WO000000489092…","""MEGA PLAZA 2 -…","""Zona Centro""","""Cerrado""",2023-01-14,2023-01-16,2023-01-19,"""MEGA PLAZA 2 """,191040,2023-01-16,2
"""WO000000524014…","""MUSA - 193050""","""Zona Sur""","""Cerrado""",2023-05-18,2023-05-23,2023-05-27,"""MUSA """,193050,2023-05-23,5
"""WO000000558727…","""ANDAHUAYLAS - …","""Zona Norte""","""Cerrado""",2023-09-09,2023-09-22,2023-09-25,"""ANDAHUAYLAS """,205000,2023-09-22,13
"""WO000000533284…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-06-19,2023-07-03,2023-07-06,"""CHIMPU OCLLO """,191113,2023-07-03,14
"""WO000000529557…","""LA ENSENADA - …","""Zona Centro""","""Cerrado""",2023-06-08,2023-06-08,2023-06-11,"""LA ENSENADA """,191149,2023-06-08,0
"""WO000000480912…","""EL AGUSTINO - …","""Zona Centro""","""Cerrado""",2022-12-13,2022-12-18,2022-12-21,"""EL AGUSTINO """,191041,2022-12-18,5
"""WO000000554863…","""VENEZUELA - 19…","""Zona Sur""","""Cerrado""",2023-08-28,2023-09-02,2023-09-05,"""VENEZUELA """,191026,2023-09-02,5
"""WO000000567330…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2023-10-07,2023-10-11,2023-10-14,"""CHIMPU OCLLO """,191113,2023-10-11,4
"""WO000000545373…","""CAQUETA - 1910…","""Zona Centro""","""Cerrado""",2023-07-31,2023-07-31,2023-08-03,"""CAQUETA """,191091,2023-07-31,0


In [16]:
condiciones_dias_cierre = [
    
    (pl.col("Dias Cierre").is_null(), None),
    (pl.col("Dias Cierre") <= 3, pl.lit("0 a 3 días")),
    (pl.col("Dias Cierre") <= 7, pl.lit("4 a 7 días")),
    (pl.col("Dias Cierre") <= 15, pl.lit("8 a 15 días")),
    (pl.col("Dias Cierre") > 15, pl.lit("+15 días")),
]

df_tickets = df_tickets.with_columns(
    pl.when(condiciones_dias_cierre[0][0]).then(condiciones_dias_cierre[0][1])
    .when(condiciones_dias_cierre[1][0]).then(condiciones_dias_cierre[1][1])
    .when(condiciones_dias_cierre[2][0]).then(condiciones_dias_cierre[2][1])
    .when(condiciones_dias_cierre[3][0]).then(condiciones_dias_cierre[3][1])
    .when(condiciones_dias_cierre[4][0]).then(condiciones_dias_cierre[4][1])
    # Agrega más condiciones según sea necesario
    .otherwise(None).alias("Grupo Dias")
)
df_tickets.sort("TicketID")

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre,Grupo Dias
str,str,str,str,date,date,date,str,i64,date,i64,str
"""OT000000484937…","""NAYLAMP - 1911…","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-25,2023-01-28,"""NAYLAMP """,191143,2023-01-25,25,"""+15 días"""
"""OT000000484938…","""LOS DAMASCOS -…","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-04,2023-01-07,"""LOS DAMASCOS """,193038,2023-01-04,4,"""4 a 7 días"""
"""OT000000484972…","""LUREN - 380004…","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-02,2023-01-05,"""LUREN """,380004,2023-01-02,2,"""0 a 3 días"""
"""OT000000484973…","""EL PINO - 1910…","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-06,2023-01-09,"""EL PINO """,191087,2023-01-06,6,"""4 a 7 días"""
"""SA000000484930…","""TUSILAGOS - 19…","""Zona Centro""","""Rechazado""",2022-12-31,null,null,"""TUSILAGOS """,191108,null,null,null
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-10,2022-06-10,"""IMPERIAL """,255001,2022-06-10,9,"""8 a 15 días"""
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-17,2022-06-17,"""IMPERIAL """,255001,2022-06-17,16,"""+15 días"""
"""WO000000412235…","""PLAZA UNION - …","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-05,2022-06-05,"""PLAZA UNION """,191062,2022-06-05,4,"""4 a 7 días"""
"""WO000000412236…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-13,2022-06-13,"""AREQUIPA """,215000,2022-06-13,12,"""8 a 15 días"""


In [17]:
# %pip install xlsx2csv

In [18]:
import os
from xlsx2csv import Xlsx2csv
#Importar excels de manera masiva?
dataframes_xlsx = []
directorio="./Atenciones"
#excelss=pl.read_excel(source="C:/Users/ycruz003/Desktop/pythonETL/Gestion Atenciones/Atenciones/Atenciones_Centro.xlsx")
dtypes = {
    "Numero Ticket": pl.Utf8,
    "Tipo de Ticket": pl.Utf8,
    "Proveedor": pl.Utf8,  # Agregar la columna "COSTO CERO" y su tipo de dato
    "Costo Atencion":pl.Utf8
}
null_values = ["", "NA", "N/A", "null", "NaN", "None", "COSTO CERO"]

for archivo in os.listdir(directorio):
    if archivo.endswith('.xlsx'):
        # Ruta completa del archivo Excel
        ruta_excel = os.path.join(directorio, archivo)
        # Nombre del archivo CSV (mismo nombre pero con extensión .csv)
        nombre_csv = os.path.splitext(archivo)[0] + '.csv'
        # Ruta completa del archivo CSV
        ruta_csv = os.path.join(directorio, nombre_csv)
        # Convertir el archivo Excel a CSV con xlsx2csv
        Xlsx2csv(ruta_excel, outputencoding="utf-8").convert(ruta_csv)
        # Leer el archivo CSV con Polars
        df_csv = pl.read_csv(ruta_csv, columns=["Numero Ticket", "Tipo de Ticket", "Proveedor","Costo Atencion"],dtypes=dtypes, ignore_errors=True, null_values=null_values)
        # Agregar el DataFrame a la lista
        dataframes_xlsx.append(df_csv)

# Concatenar todos los DataFrames en uno solo
df_atenciones = pl.concat(dataframes_xlsx)
df_atenciones


Numero Ticket,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,str
"""WO000000558468…","""Variable""","""MR EXPRESS""",null
"""WO000000454494…","""Rechazado""",null,"""100"""
"""WO000000454524…","""Rechazado""",null,"""100"""
"""WO000000455912…","""Rechazado""",null,"""100"""
"""WO000000470747…","""Rechazado""",null,"""100"""
"""WO000000475997…","""Rechazado""",null,"""100"""
"""WO000000479587…","""Rechazado""",null,"""100"""
"""WO000000482530…","""Rechazado""",null,"""100"""
"""WO000000485368…","""Flat""","""ACCENTURE""","""100"""


In [19]:
# Cambiar el tipo
df_atenciones=df_atenciones.select( pl.col("Numero Ticket").alias("TicketID"), *df_atenciones.drop("Numero Ticket"))
df_atenciones.sort("TicketID")

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,str
"""WO000000412230…","""Flat""","""ACCENTURE""",null
"""WO000000412230…","""Flat""","""ACCENTURE""",null
"""WO000000412235…","""Flat""","""ACCENTURE""",null
"""WO000000412236…","""Flat""","""ACCENTURE""",null
"""WO000000412238…","""Rechazado""",null,null
"""WO000000412240…","""Flat""","""ACCENTURE""",null
"""WO000000412241…","""Rechazado""",null,null
"""WO000000412242…","""Variable""","""PROSEGUR""",null
"""WO000000412244…","""Variable""","""PROSEGUR""",null


In [20]:
# Uppercase a Costo Atencion
df_atenciones=df_atenciones.with_columns(pl.col("Costo Atencion").str.to_uppercase())

df_atenciones=df_atenciones.with_columns(pl.col("Costo Atencion").str.replace(',','.'))

df_atenciones = df_atenciones.with_columns(
    (pl.when(df_atenciones['Costo Atencion'] == 'SIN COSTO').then(pl.lit('0')).otherwise(df_atenciones['Costo Atencion'])).alias("Costo Atencion"),
    
)
df_atenciones = df_atenciones.with_columns(
    (pl.when(df_atenciones['Costo Atencion'] == 'COSTO CERO').then(pl.lit('0')).otherwise(df_atenciones['Costo Atencion'])).alias("Costo Atencion")
)

# Convertir a decimal Costo Atencion
df_atenciones=df_atenciones.cast({"Costo Atencion":pl.Float64}, strict=False)
df_atenciones

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,f64
"""WO000000558468…","""Variable""","""MR EXPRESS""",null
"""WO000000454494…","""Rechazado""",null,100.0
"""WO000000454524…","""Rechazado""",null,100.0
"""WO000000455912…","""Rechazado""",null,100.0
"""WO000000470747…","""Rechazado""",null,100.0
"""WO000000475997…","""Rechazado""",null,100.0
"""WO000000479587…","""Rechazado""",null,100.0
"""WO000000482530…","""Rechazado""",null,100.0
"""WO000000485368…","""Flat""","""ACCENTURE""",100.0


In [21]:
df_atenciones=df_atenciones.group_by("*").agg(pl.col("*").unique())
df_atenciones

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,f64
"""WO000000454524…","""Rechazado""",null,100.0
"""WO000000506408…","""Flat""","""ACCENTURE""",100.0
"""WO000000554905…","""Flat""","""ACCENTURE""",100.0
"""WO000000568856…","""Variable""","""CECHRIZA""",1019.28
"""WO000000493583…","""Variable""","""JGM IMPORT""",102.66
"""WO000000558518…","""Variable""","""COORPORACION R…",1026.79
"""WO000000557375…","""Emergencia""","""PUNTO CONTROL …",1028.37
"""WO000000558751…","""Flat""","""ACCENTURE""",104.5
"""WO000000520072…","""Emergencia""","""PUNTO CONTROL …",1044.3


In [22]:
#Juntar los dos dataframes
df_final=df_atenciones.join(df_tickets, on="TicketID", how="inner")
df_final

TicketID,Tipo de Ticket,Proveedor,Costo Atencion,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre,Grupo Dias
str,str,str,f64,str,str,str,date,date,date,str,i64,date,i64,str
"""WO000000454524…","""Rechazado""",null,100.0,"""PUENTE PIEDRA …","""Zona Centro""","""Rechazado""",2022-10-18,2022-10-18,null,"""PUENTE PIEDRA …",191035,2022-10-18,0,"""0 a 3 días"""
"""WO000000506408…","""Flat""","""ACCENTURE""",100.0,"""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-03-09,2023-03-18,2023-03-21,"""EL SOL """,191082,2023-03-18,9,"""8 a 15 días"""
"""WO000000554905…","""Flat""","""ACCENTURE""",100.0,"""MEGA PLAZA - 1…","""Zona Centro""","""Cerrado""",2023-08-28,2023-09-01,2023-09-04,"""MEGA PLAZA """,191010,2023-09-01,4,"""4 a 7 días"""
"""WO000000568856…","""Variable""","""CECHRIZA""",1019.28,"""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-10-12,2023-10-21,2023-10-24,"""SAN MARTIN DE …",191021,2023-10-21,9,"""8 a 15 días"""
"""WO000000493583…","""Variable""","""JGM IMPORT""",102.66,"""DEL AIRE - 193…","""Zona Centro""","""Cerrado""",2023-01-28,2023-02-04,2023-02-07,"""DEL AIRE """,193064,2023-02-04,7,"""4 a 7 días"""
"""WO000000558518…","""Variable""","""COORPORACION R…",1026.79,"""DEL AIRE - 193…","""Zona Centro""","""Cerrado""",2023-09-09,2023-09-15,2023-09-18,"""DEL AIRE """,193064,2023-09-15,6,"""4 a 7 días"""
"""WO000000557375…","""Emergencia""","""PUNTO CONTROL …",1028.37,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""",2023-09-04,2023-09-21,2023-09-24,"""PLAZA LIMA NOR…",191058,2023-09-21,17,"""+15 días"""
"""WO000000558751…","""Flat""","""ACCENTURE""",104.5,"""NACIONES UNIDA…","""Zona Centro""","""Cerrado""",2023-09-09,2023-09-14,2023-09-17,"""NACIONES UNIDA…",191084,2023-09-14,5,"""4 a 7 días"""
"""WO000000520072…","""Emergencia""","""PUNTO CONTROL …",1044.3,"""ZARATE - 19109…","""Zona Centro""","""Cerrado""",2023-05-04,2023-05-20,2023-05-23,"""ZARATE """,191097,2023-05-20,16,"""+15 días"""


In [23]:
# Definir la estructurade variables del df
df_final=df_final.select("TicketID","AgenciaID","Agencia","Service Desk","Estado",pl.col("Fecha Creacion").dt.strftime("%d/%m/%Y"),pl.col("Fecha Real Fin").dt.strftime("%d/%m/%Y").alias("Fecha Cierre"),"Grupo Dias",pl.col("Tipo de Ticket").alias("Tipo Ticket"), pl.col("Costo Atencion").round(2).alias("Costo"))
df_final

TicketID,AgenciaID,Agencia,Service Desk,Estado,Fecha Creacion,Fecha Cierre,Grupo Dias,Tipo Ticket,Costo
str,i64,str,str,str,str,str,str,str,f64
"""WO000000454524…",191035,"""PUENTE PIEDRA …","""Zona Centro""","""Rechazado""","""18/10/2022""","""18/10/2022""","""0 a 3 días""","""Rechazado""",100.0
"""WO000000506408…",191082,"""EL SOL ""","""Zona Centro""","""Cerrado""","""09/03/2023""","""18/03/2023""","""8 a 15 días""","""Flat""",100.0
"""WO000000554905…",191010,"""MEGA PLAZA ""","""Zona Centro""","""Cerrado""","""28/08/2023""","""01/09/2023""","""4 a 7 días""","""Flat""",100.0
"""WO000000568856…",191021,"""SAN MARTIN DE …","""Zona Centro""","""Cerrado""","""12/10/2023""","""21/10/2023""","""8 a 15 días""","""Variable""",1019.28
"""WO000000493583…",193064,"""DEL AIRE ""","""Zona Centro""","""Cerrado""","""28/01/2023""","""04/02/2023""","""4 a 7 días""","""Variable""",102.66
"""WO000000558518…",193064,"""DEL AIRE ""","""Zona Centro""","""Cerrado""","""09/09/2023""","""15/09/2023""","""4 a 7 días""","""Variable""",1026.79
"""WO000000557375…",191058,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""","""04/09/2023""","""21/09/2023""","""+15 días""","""Emergencia""",1028.37
"""WO000000558751…",191084,"""NACIONES UNIDA…","""Zona Centro""","""Cerrado""","""09/09/2023""","""14/09/2023""","""4 a 7 días""","""Flat""",104.5
"""WO000000520072…",191097,"""ZARATE ""","""Zona Centro""","""Cerrado""","""04/05/2023""","""20/05/2023""","""+15 días""","""Emergencia""",1044.3


In [24]:
# %pip install pandas
# %pip install pyarrow
# %pip install numpy
# %pip install openpyxl

In [25]:
# Se esta utilizando pandas, y todas las lbrerias menionadas arriba puesto que polars ni xlsx2csv tienen la funcion de exportar excels
import pandas as pd
dt_final_pd= df_final.to_pandas()
dt_final_pd.to_excel(
    "./Consolidado.xlsx", sheet_name="Atenciones", index=False
)